<a href="https://colab.research.google.com/github/Esbern/conservation-agriculture/blob/main/ProcessPixels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import numpy as np


In [ ]:
# Replace 'your_github_file_url_here' with the actual URL of your JSON file on GitHub
url = 'https://raw.githubusercontent.com/Esbern/conservation-agriculture/main/ca_pixler.json'

# Fetch the JSON data
response = requests.get(url)
data = response.json()

# Assuming the first sub-array contains the column names
columns = data[0]
# And the rest of the sub-arrays contain the data
data_rows = data[1:]

# Load the data into a pandas DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# Display the first few rows of the DataFrame
print(df.head())


                     id  longitude   latitude           time        B3  \
0  LE07_194021_20120502  12.033876  55.240865  1335952945389  0.054968   
1  LE07_194021_20120518  12.033876  55.240865  1337335359149       NaN   
2  LE07_194022_20100529  12.033876  55.240865  1275127227884       NaN   
3  LE07_194022_20100614  12.033876  55.240865  1276509628532  0.259122   
4  LE07_194022_20100630  12.033876  55.240865  1277892031898  0.393416   

         B4  QA_PIXEL  
0  0.450163    5440.0  
1       NaN       NaN  
2       NaN       NaN  
3  0.468619    5896.0  
4  0.773263    5896.0  


In [ ]:
df['time'] = pd.to_datetime(df['time'], unit='ms')

# Extract year, month, and day to separate columns
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day

# Display the DataFrame to verify the changes
print(df.head())

                     id  longitude   latitude                    time  \
0  LE07_194021_20120502  12.033876  55.240865 2012-05-02 10:02:25.389   
1  LE07_194021_20120518  12.033876  55.240865 2012-05-18 10:02:39.149   
2  LE07_194022_20100529  12.033876  55.240865 2010-05-29 10:00:27.884   
3  LE07_194022_20100614  12.033876  55.240865 2010-06-14 10:00:28.532   
4  LE07_194022_20100630  12.033876  55.240865 2010-06-30 10:00:31.898   

         B3        B4  QA_PIXEL  year  month  day  
0  0.054968  0.450163    5440.0  2012      5    2  
1       NaN       NaN       NaN  2012      5   18  
2       NaN       NaN       NaN  2010      5   29  
3  0.259122  0.468619    5896.0  2010      6   14  
4  0.393416  0.773263    5896.0  2010      6   30  


In [ ]:

# Sample DataFrame creation with NaN values (for demonstration)
# data = {'QA_PIXEL': [5440.0, np.nan, 2, 4, np.nan, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]}
# df = pd.DataFrame(data)

# Function to decode the QA_PIXEL column, modified to handle NaN values
def decode_qa_pixel(value):
    # Check if the value is NaN and return NaN for all attributes if true
    if pd.isna(value):
        return pd.Series([np.nan] * 11, index=[
            'Fill', 'Dilated Cloud', 'Cloud', 'Cloud Shadow', 'Snow',
            'Clear', 'Water', 'Cloud Confidence', 'Cloud Shadow Confidence',
            'Snow/Ice Confidence', 'Cirrus Confidence'])

    # Convert float to int and then to binary string, stripping the '0b' prefix
    binary_str = format(int(value), '016b')

    # Decode each bit based on the descriptions provided
    attributes = {
        'Fill': 'Fill data' if binary_str[-1] == '1' else 'Image data',
        'Dilated Cloud': 'Cloud dilation' if binary_str[-2] == '1' else 'Cloud is not dilated or no cloud',
        'Cloud': 'High confidence cloud' if binary_str[-4] == '1' else 'Cloud confidence is not high',
        'Cloud Shadow': 'High confidence cloud shadow' if binary_str[-5] == '1' else ' Cloud Shadow Confidence is not high',
        'Snow': 'High confidence snow cover' if binary_str[-6] == '1' else 'Snow/Ice Confidence is not high',
        'Clear': 'Cloud and Dilated Cloud bits not set' if binary_str[-7] == '1' else 'Cloud or Dilated Cloud bits set',
        'Water': 'Water' if binary_str[-8] == '1' else 'Land or cloud',
        'Cloud Confidence': {
            '00': 'No cloud confidence level set',
            '01': 'Low cloud confidence',
            '10': 'Medium cloud confidence',
            '11': 'High cloud confidence',
        }[binary_str[-10:-8]],
        'Cloud Shadow Confidence': {
            '00': 'No cloud shadow confidence level set',
            '01': 'Low cloud shadow confidence',
            '10': 'Reserved',
            '11': 'High cloud shadow confidence',
        }[binary_str[-12:-10]],
        'Snow/Ice Confidence': {
            '00': 'No snow/ice confidence level set',
            '01': 'Low snow/ice confidence',
            '10': 'Reserved',
            '11': 'High snow/ice confidence',
        }[binary_str[-14:-12]],
        'Cirrus Confidence': {
            '00': 'No cirrus confidence level set',
            '01': 'Low cirrus confidence',
            '10': 'Reserved',
            '11': 'High cirrus confidence',
        }[binary_str[-16:-14]],
    }

    return pd.Series(attributes)

# Apply the decode function to the QA_PIXEL column and assign the result to new columns
new_columns = df['QA_PIXEL'].apply(decode_qa_pixel)
df = pd.concat([df, new_columns], axis=1)

# Display the DataFrame to verify the changes
print(df.head())


                     id  longitude   latitude                    time  \
0  LE07_194021_20120502  12.033876  55.240865 2012-05-02 10:02:25.389   
1  LE07_194021_20120518  12.033876  55.240865 2012-05-18 10:02:39.149   
2  LE07_194022_20100529  12.033876  55.240865 2010-05-29 10:00:27.884   
3  LE07_194022_20100614  12.033876  55.240865 2010-06-14 10:00:28.532   
4  LE07_194022_20100630  12.033876  55.240865 2010-06-30 10:00:31.898   

         B3        B4  QA_PIXEL  year  month  day  ...  \
0  0.054968  0.450163    5440.0  2012      5    2  ...   
1       NaN       NaN       NaN  2012      5   18  ...   
2       NaN       NaN       NaN  2010      5   29  ...   
3  0.259122  0.468619    5896.0  2010      6   14  ...   
4  0.393416  0.773263    5896.0  2010      6   30  ...   

                   Dilated Cloud                      Cloud  \
0  No cloud dilation or no cloud  Not high confidence cloud   
1                            NaN                        NaN   
2                      

In [ ]:
dilated_cloud_counts = df['Cirrus Confidence'].value_counts()
dilated_cloud_counts

No cirrus confidence level set    9109
Name: Cirrus Confidence, dtype: int64

In [ ]:
print(df.columns)

Index(['id', 'longitude', 'latitude', 'time', 'B3', 'B4', 'QA_PIXEL', 'year',
       'month', 'day', 'Fill', 'Dilated Cloud', 'Cloud', 'Cloud Shadow',
       'Snow', 'Clear', 'Water', 'Cloud Confidence', 'Cloud Shadow Confidence',
       'Snow/Ice Confidence', 'Cirrus Confidence'],
      dtype='object')


In [ ]:
unique_combinations = df[['longitude', 'latitude']].drop_duplicates()

In [ ]:
unique_combinations

,longitude,latitude
0,12.033876,55.240865
127,12.034146,55.240865
254,12.034415,55.240865
381,12.034685,55.240865
508,12.031182,55.241135
...,...,...
15462,11.480065,55.278325
15685,11.480335,55.278325
15908,11.480604,55.278325
16131,11.480874,55.278325


In [ ]:
unique_combinations.to_excel(file_path, index=False, engine='openpyxl')

In [ ]:
# Assuming unique_combinations is your DataFrame
file_path = '/content/drive/My Drive/unique_combinations.xlsx'  # Adjust the path as needed

# Export the DataFrame to an Excel file in Google Drive
unique_combinations.to_excel(file_path, index=False, engine='openpyxl')

print(f'The unique combinations have been saved to Google Drive at {file_path}')

The unique combinations have been saved to Google Drive at /content/drive/My Drive/unique_combinations.xlsx
